#### Sentence Transformation and Embeddings

#### Acknowledgements

- https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2

#### Packages

In [1]:
import sentence_transformers as pkg_sentence_transformers
import scipy.spatial as pkg_spatial

#### Common

In [2]:
model_path = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
model = pkg_sentence_transformers.SentenceTransformer(model_path)

In [3]:
cosine_similarity = lambda x,y: (1 - pkg_spatial.distance.cosine(x,y))

#### Work

In [4]:
# Reference Sentence
ref_sentence = "That is a happy person"

##### Find Similiarity

In [5]:
# Sentences we want compare with
cmp_sentences = [
    "Sunny days make people happy",
    "Rainy days make people happy"
    "Relationships enhance happiness",
    "There is no happy person in the world. Happiness is misnomer",
    "Mind free from tensions make a person happy"
]

ref_embeddings = model.encode(sentences=[ref_sentence])
cmp_embeddings = model.encode(cmp_sentences)

print ("Reference Sentence: {}".format(ref_sentence))
for i in range(len(cmp_embeddings)):
    similiarity = cosine_similarity(ref_embeddings[0], cmp_embeddings[i])
    print("Similiarity: {} for sentence = {}".format(similiarity, cmp_sentences[i]))

Reference Sentence: That is a happy person
Similiarity: 0.6686985492706299 for sentence = Sunny days make people happy
Similiarity: 0.5185865163803101 for sentence = Rainy days make people happyRelationships enhance happiness
Similiarity: 0.48634278774261475 for sentence = There is no happy person in the world. Happiness is misnomer
Similiarity: 0.5353088974952698 for sentence = Mind free from tensions make a person happy
